# Text to speech API

## Install Google Cloud Text-to-speech

In [60]:
#!pip install --upgrade google-cloud-texttospeech
#!pip install --upgrade google-cloud-storage
#!pip install cloudstorage
from google.cloud import texttospeech
from google.oauth2 import service_account
import IPython.display as ipd
import html

## Google service account credentials

In [4]:
file_credentials = "Kanban-7b2a3d12dc04.json"
credentials = service_account.Credentials. from_service_account_file(file_credentials)

## Text to SSML function

In [61]:
def text_to_ssml(text):
    # Generates SSML text from plaintext
    # Arguments: inputfile - string name of plaintext file
    # Returns:A string of SSML text based on plaintext input

# Replace special characters with HTML Ampersand Character Codes
    # These Codes prevent the API from confusing text with
    # SSML commands
    # For example, '<' --> '&lt;' and '&' --> '&amp;'

    escaped_lines = html.escape(text)

    # Convert plaintext to SSML
    # Wait two seconds between each address
    ssml = "<speak><prosody>{}</prosody></speak>".format(
        escaped_lines.replace("\n", '\n<break time="400ms"/>')
    )
    
    return ssml

## Text to speech

### Voice

In [62]:
voice = texttospeech.types.VoiceSelectionParams(
        language_code="en-GB",
        name="en-GB-Wavenet-A",
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE,
    )

### Function

In [7]:
def synthesize_text(a, voice):

    client = texttospeech.TextToSpeechClient(credentials=credentials)
    
    synthesis_input = texttospeech.types.SynthesisInput(ssml=a)

    print(synthesis_input)
    
    audio_config = texttospeech.types.AudioConfig(
        audio_encoding=texttospeech.enums.AudioEncoding.LINEAR16,
        speaking_rate=0.90,
        pitch=-2
    )

    print(audio_config)

    response = client.synthesize_speech(
        synthesis_input, voice, audio_config
    )

    with open('output.wav', 'wb') as out:
    # Write the response to the output file.
        out.write(response.audio_content)
        print('Audio content written to file "output.wav"')
    out.close()

### Read Text file from Google Cloud Storage Bucket

In [72]:
text_in_gcs='example.txt'   # File containing the text to pass in the text-to-speech function

In [73]:
from google.datalab import Context
import google.datalab.storage as storage

project_id=Context.default().project_id
folder_bucket_name = project_id + '-output'
folder_bucket = storage.Bucket(folder_bucket_name)

if not folder_bucket.exists():
    folder_bucket.create()

text_file=folder_bucket.object(text_in_gcs)

contents=text_file.read_stream().decode('utf-8')
print(contents)

We, the People, recognize that we have responsibilities as well as rights; 
that our destinies are bound together; 
that a freedom which only asks what's in it for me, a freedom without a commitment to others, 
a freedom without love or charity or duty or patriotism, is unworthy of our founding ideals, and those who died in their defense.


### Run text to speech function

In [74]:
a = text_to_ssml(contents)
synthesize_text(a, voice)

ssml: "<speak><prosody>We, the People, recognize that we have responsibilities as well as rights; \r\n<break time=\"400ms\"/>that our destinies are bound together; \r\n<break time=\"400ms\"/>that a freedom which only asks what&#x27;s in it for me, a freedom without a commitment to others, \r\n<break time=\"400ms\"/>a freedom without love or charity or duty or patriotism, is unworthy of our founding ideals, and those who died in their defense.</prosody></speak>"

audio_encoding: LINEAR16
speaking_rate: 0.9
pitch: -2.0

Audio content written to file "output.wav"


## Listening!

In [75]:
ipd.Audio("output.wav")

### Copy audio file to GCS Bucket

In [105]:
!gsutil cp output.wav gs://kanban-290909-output/output1.wav

Copying file://output.wav [Content-Type=audio/x-wav]...
/ [1 files][976.2 KiB/976.2 KiB]                                                
Operation completed over 1 objects/976.2 KiB.                                    


### Play the GCS Bucket file just created

In [106]:
audio_file=folder_bucket.object("output1.wav")
ipd.Audio(audio_file.read_stream())